In [15]:
import numpy as np

In [16]:
mesh_vertices = np.load('data/scannet/pointgroup_data/scene0568_00_aligned_vert.npy')

In [17]:
instance_labels = np.load('data/scannet/pointgroup_data/scene0568_00_ins_label_pg.npy')

In [18]:
semantic_labels = np.load('data/scannet/pointgroup_data/scene0568_00_sem_label_pg.npy')

In [29]:
# semantic_labels = np.load('data/scannet/pointgroup_data/scene0032_00_sem_label_pg.npy')
instance_bboxes = np.load('data/scannet/pointgroup_data/scene0568_00_aligned_bbox.npy')

In [30]:
_instance_bboxes = np.load('data/scannet/pointgroup_data/scene0568_00_bbox.npy')

In [31]:
instance_bboxes[0]

array([2.35776114, 1.78411531, 0.22498837, 0.55324221, 0.49477351,
       0.64744705, 5.        , 0.        ])

In [32]:
_instance_bboxes[0]

array([0.51633686, 1.93914962, 1.03231835, 0.55899203, 0.49096203,
       0.64744705, 5.        , 0.        ])

In [14]:
import open3d as o3d

# Create a sample point cloud (random points)
points = mesh_vertices[:, 0:3]
pcl_color = mesh_vertices[:, 3:6]/255.0
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(pcl_color)

# Define multiple axis-aligned bounding boxes
aabbs = []
for box in instance_bboxes:
    center = box[0:3].tolist()
    size = box[3:6].tolist()
    min_bound = np.array(center) - np.array(size) / 2
    max_bound = np.array(center) + np.array(size) / 2
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
    aabb.color = (0, 1, 0)  # green
    aabbs.append(aabb)

# Visualize point cloud with bounding boxes
o3d.visualization.draw_geometries([pcd] + aabbs)

In [33]:
import torch

In [34]:
a = torch.zeros((4, 128))

In [35]:
torch.sum(a)

tensor(0.)